In [1]:
import pandas as pd
import math
import numpy as np
import glob
import collections
import re
from datetime import datetime

In [2]:
NAMESPACE = "https://iisg.amsterdam/"
DATASET_NAME = "links/"

PREFIX_IISG = NAMESPACE + DATASET_NAME
PREFIX_IISG_VOCAB = PREFIX_IISG + "vocab/"
PREFIX_SCHEMA = "http://schema.org/"
PREFIX_BIO = "http://purl.org/vocab/bio/0.1/"
PREFIX_DC = "http://purl.org/dc/terms/"

TYPE_BIRTH_REGISTRATION = PREFIX_IISG_VOCAB + "BirthRegistration"
TYPE_BIRTH_EVENT = PREFIX_BIO + "Birth"
MARRIAGE_REGISTRATION = PREFIX_IISG_VOCAB + "MarriageRegistration"
TYPE_MARRIAGE_EVENT = PREFIX_BIO + "Marriage"
TYPE_DEATH_REGISTRATION = PREFIX_IISG_VOCAB + "DeathRegistration"
TYPE_DEATH_EVENT = PREFIX_BIO + "Death"
TYPE_DIVORCE_REGISTRATION = PREFIX_IISG_VOCAB + "DivorceRegistration"
TYPE_PERSON = PREFIX_SCHEMA + "Person"
TYPE_PLACE = PREFIX_SCHEMA + "Place"
TYPE_COUNTRY = PREFIX_IISG + "Country"
TYPE_REGION = PREFIX_IISG + "Region"
TYPE_PROVINCE = PREFIX_IISG + "Province"
TYPE_MUNICIPALITY = PREFIX_IISG + "Municipality"

RDF_TYPE = "http://www.w3.org/1999/02/22-rdf-syntax-ns#type"
OWL_SAMEAS = "http://www.w3.org/2002/07/owl#sameAs"
REGISTER_EVENT = PREFIX_IISG_VOCAB + "registerEvent"
LOCATION = PREFIX_SCHEMA + "location"
DATE = PREFIX_BIO + "date"
REGISTRATION_ID = PREFIX_IISG_VOCAB + "registrationID"
REGISTRATION_SEQUENCE = PREFIX_IISG_VOCAB + "registrationSeqID"
BIRTH_DATE_FLAG = PREFIX_IISG_VOCAB + "birthDateFlag"

PERSON_ID = PREFIX_IISG_VOCAB + "personID"
GIVEN_NAME = PREFIX_SCHEMA + "givenName"
FAMILY_NAME = PREFIX_SCHEMA + "familyName"
GENDER = PREFIX_SCHEMA + "gender"
OCCUPATION = PREFIX_SCHEMA + "hasOccupation"
FAMILY_NAME_PREFIX = PREFIX_IISG_VOCAB + "prefixFamilyName"
AGE = PREFIX_IISG_VOCAB + "ageOfMarriage"

ROLE_NEWBORN = PREFIX_IISG_VOCAB + "newborn"
ROLE_MOTHER = PREFIX_IISG_VOCAB + "mother"
ROLE_FATHER = PREFIX_IISG_VOCAB + "father"
ROLE_DECEASED = PREFIX_IISG_VOCAB + "deceased"
ROLE_DECEASED_PARTNER = PREFIX_IISG_VOCAB + "deceasedPartner"
ROLE_BRIDE = PREFIX_IISG_VOCAB + "bride"
ROLE_BRIDE_MOTHER = PREFIX_IISG_VOCAB + "motherBride"
ROLE_BRIDE_FATHER = PREFIX_IISG_VOCAB + "fatherBride"
ROLE_GROOM = PREFIX_IISG_VOCAB + "groom"
ROLE_GROOM_MOTHER = PREFIX_IISG_VOCAB + "motherGroom"
ROLE_GROOM_FATHER = PREFIX_IISG_VOCAB + "fatherGroom"
ROLE_UNKNOWN = PREFIX_IISG_VOCAB + "unknown"

In [3]:
def isNaN(num):
    if num == "\\N" or num == "":
        return True
    else:
        return False

def isNaN_Number(num):
    if num == 0:
        return True
    else:
        return False

def transformToInt(someNumber):
    return '"' + str(someNumber) + '"^^<http://www.w3.org/2001/XMLSchema#int>'

def transformToString(someWord):
    return '"' + str(someWord) + '"^^<http://www.w3.org/2001/XMLSchema#string>'

def transformToDate(someDate):
    return '"' + str(someDate) + '"^^<http://www.w3.org/2001/XMLSchema#date>'

def createQuad(s, p, o):
    return s + ' ' + p + ' ' + o + ' <https://iisg.amsterdam/links/marriages> .\n'

def getRole(role_number, registrationID):
    if str(role_number) == '4':
        return ROLE_BRIDE
    if str(role_number) == '5':
        return ROLE_BRIDE_MOTHER
    if str(role_number) == '6':
        return ROLE_BRIDE_FATHER
    if str(role_number) == '7':
        return ROLE_GROOM
    if str(role_number) == '8':
        return ROLE_GROOM_MOTHER
    if str(role_number) == '9':
        return ROLE_GROOM_FATHER
    else:
        print("Role number:", role_number, "- Certificate ID:", registrationID)
        return "UNKNOWN"

In [4]:
def createRegistrationURI(registrationID):
    return "<" + PREFIX_IISG + "registration/" + (str(registrationID)) + ">"

def createPersonURI(personID):
    return "<" + PREFIX_IISG + "person/" + (str(personID)) + ">"

def createEventURI(registrationID):
    return "<" + PREFIX_IISG + "event/" + (str(registrationID)) + ">"

def createTripleRegistrationID(registrationURI, registrationID):
    p = "<" + PREFIX_IISG_VOCAB + "registrationID" + ">"
    o = transformToInt(registrationID)
    return createQuad(registrationURI,p,o)

def createRegistrationTypeURI(registrationType):
    if registrationType == 'g':
        return "<" + PREFIX_IISG_VOCAB + "BirthRegistration" + ">"
    if registrationType == 'h':
        return "<" + PREFIX_IISG_VOCAB + "MarriageRegistration" + ">"
    if registrationType == 'o':
        return "<" + PREFIX_IISG_VOCAB + "DeathRegistration" + ">"
    if registrationType == 's':
        return "<" + PREFIX_IISG_VOCAB + "DivorceRegistration" + ">"
    else:
        return "<" + PREFIX_IISG_VOCAB + "MarriageRegistration" + ">"
    
def createTripleRegistrationType(registrationURI, registrationType):
    p = "<" + RDF_TYPE + ">"
    o = createRegistrationTypeURI(registrationType)
    if o != "EMTPY":
        return createQuad(registrationURI, p, o)
    else:
        print("Something is wrong", registrationURI, registrationType)
    
def createDate(year, month, day):
    fixedYear = str(year).zfill(4)
    fixedMonth = str(month).zfill(2)
    fixedDay = str(day).zfill(2)
    return transformToDate(fixedYear + "-" + fixedMonth + "-" + fixedDay)

def createTripleDate(registrationURI, fixedDate):
    p = "<" + DATE + ">"
    return createQuad(registrationURI, p , fixedDate)

def createLocationURI(locationID):
    return "<" + PREFIX_IISG + "place/" + str(locationID) + ">"
    
def createTripleLocation(registrationURI, locationID):
    p = "<" + LOCATION + ">"
    o = createLocationURI(locationID)
    return createQuad(registrationURI,p ,o)

def createTripleRegistrationSeq(registrationURI, registrationSeqID):
    p = "<" + PREFIX_IISG_VOCAB + "registrationSeqID" + ">"
    o = transformToString(registrationSeqID)
    return createQuad(registrationURI, p, o)

def createTripleRegistrationOrigID(registrationURI, registrationOrigID):
    p = "<" + PREFIX_IISG_VOCAB + "registrationOriginalID" + ">"
    o = transformToInt(registrationOrigID)
    return createQuad(registrationURI, p, o)

def createTripleLinksBase(registrationURI, not_linksbase):
    p = "<" + PREFIX_IISG_VOCAB + "linksBase" + ">"
    o = transformToString(not_linksbase)
    return createQuad(registrationURI, p, o)

def createTriplesRegisterEvent(registrationURI, eventURI):
    p = "<" + REGISTER_EVENT + ">"
    return createQuad(registrationURI, p, eventURI)

def createTriplePersonType(personURI):
    p = "<" + RDF_TYPE + ">"
    o = "<" + TYPE_PERSON + ">"
    return createQuad(personURI, p, o)

def createTriplePersonID(personURI, personID):
    p = "<" + PREFIX_IISG_VOCAB + "personID" + ">"
    o = transformToInt(personID)
    return createQuad(personURI, p, o)

def createTripleGender(personURI, gender):
    p = "<" + GENDER + ">"
    o = transformToString(gender)
    return createQuad(personURI, p, o)

def createTripleGivenName(personURI, givenName):
    p = "<" + GIVEN_NAME + ">"
    o = transformToString(givenName)
    return createQuad(personURI, p, o)

def createTripleFamilyName(personURI, familyName):
    p = "<" + FAMILY_NAME + ">"
    o = transformToString(familyName)
    return createQuad(personURI, p, o)

def createTriplePrefix(personURI, prefix):
    p = "<" + FAMILY_NAME_PREFIX + ">"
    o = transformToString(prefix)
    return createQuad(personURI, p, o)

def createTripleOccupation(personURI, occupation):
    p = "<" + OCCUPATION + ">"
    o = transformToString(occupation)
    return createQuad(personURI, p, o)

def createTripleAge(personURI, age):
    p = "<" + AGE + ">"
    o = transformToInt(age)
    return createQuad(personURI, p, o)

def createTripleRole(eventURI, normalisedRole, personURI):
    p = "<" + normalisedRole + ">"
    return createQuad(eventURI, p, personURI)

In [5]:
def convertRegistrationsToRDF(inputData, outputData):
    print("Converting Registrations CSV file to RDF...")
    start_time = datetime.now() 
    f = open(outputData,"w+")
    ch_size = 10000
    df_chunk = pd.read_csv(inputData, chunksize=ch_size, low_memory=False, keep_default_na=False)
    counter = 0
    for chunk in df_chunk:
        print("# " + str(counter) + " rows")
        counter = counter + ch_size
        filebuffer = []
        for index, row in chunk.iterrows():
            registrationID = row['id_registration']
            registrationURI = createRegistrationURI(registrationID)
            if not isNaN(registrationID):
                filebuffer.append(createTripleRegistrationID(registrationURI, registrationID))
                registrationType = row['registration_type']
                if not isNaN(registrationType):
                    filebuffer.append(createTripleRegistrationType(registrationURI, registrationType))
                year = row['registration_year']
                month = row['registration_month']
                day = row['registration_day']
                if not isNaN_Number(year) and not isNaN_Number(month) and not isNaN_Number(day):
                    if not isNaN(year) and not isNaN(month) and not isNaN(day):
                        fixedDate = createDate(year, month, day)
                        filebuffer.append(createTripleDate(registrationURI, fixedDate))
                locationID = row['registration_location_no']
                if not isNaN(locationID):
                    filebuffer.append(createTripleLocation(registrationURI, locationID))
                registrationOrigID = row['id_orig_registration']
                if not isNaN(registrationOrigID):
                    filebuffer.append(createTripleRegistrationOrigID(registrationURI, registrationOrigID))
                registrationSeqID = row['registration_seq']
                if not isNaN(registrationSeqID):
                    filebuffer.append(createTripleRegistrationSeq(registrationURI, registrationSeqID))
                not_linksbase = row['not_linksbase']
                if not isNaN(not_linksbase):
                    filebuffer.append(createTripleLinksBase(registrationURI, not_linksbase))
        f.writelines(filebuffer)
    f.close()
    print("Program Finished!")
    time_elapsed = datetime.now() - start_time 
    print('Time elapsed (hh:mm:ss) {}'.format(time_elapsed))

In [6]:
def convertPersonsToRDF(inputData, outputData):
    print("Converting Persons CSV file to RDF...")
    start_time = datetime.now() 
    f = open(outputData,"w+")
    ch_size = 10000
    df_chunk = pd.read_csv(inputData, chunksize=ch_size, low_memory=False, error_bad_lines=False, keep_default_na=False)
    counter = 0
    for chunk in df_chunk:
        print("# " + str(counter) + " rows")
        counter = counter + ch_size
        filebuffer = []
        for index, row in chunk.iterrows():
            personID = row['id_person']
            registrationID = row['id_registration']
            if not isNaN(personID) and not isNaN(registrationID):
                personURI = createPersonURI(personID)
                registrationURI = createRegistrationURI(registrationID)
                eventURI = createEventURI(registrationID)        
                filebuffer.append(createTriplePersonType(personURI))
                filebuffer.append(createTriplePersonID(personURI, personID))                             
                givenName = row['firstname']
                if not isNaN(givenName):
                    filebuffer.append(createTripleGivenName(personURI, givenName))
                familyName = row['familyname']
                if not isNaN(familyName):
                    filebuffer.append(createTripleFamilyName(personURI, familyName))
                prefix = row['prefix']
                if not isNaN(prefix):
                    filebuffer.append(createTriplePrefix(personURI, prefix))    
                gender = row['sex']
                if not isNaN(gender):
                    filebuffer.append(createTripleGender(personURI, gender))
                occupation = row['occupation']
                if not isNaN(occupation):
                    occupationFixed = re.sub('[^A-Za-z0-9 ]+', '', occupation)
                    filebuffer.append(createTripleOccupation(personURI, occupationFixed))
                age = row['age_year']
                if not isNaN(age):
                    filebuffer.append(createTripleAge(personURI, age))
                role = row['role']
                if not isNaN(role):
                    normalisedRole = getRole(role, registrationID)
                    if normalisedRole != "UNKNOWN":
                        filebuffer.append(createTripleRole(eventURI, normalisedRole, personURI))
                    if normalisedRole == ROLE_BRIDE:
                        filebuffer.append(createTriplesRegisterEvent(registrationURI, eventURI))
                        year = row['mar_year']
                        month = row['mar_month']
                        day = row['mar_day']
                        if not isNaN_Number(year) and not isNaN_Number(month) and not isNaN_Number(day):
                            if not isNaN(year) and not isNaN(month) and not isNaN(day):
                                fixedDate = createDate(year, month, day)
                                filebuffer.append(createTripleDate(eventURI, fixedDate))
                        locationID = row['mar_location']
                        if not isNaN(locationID):
                            filebuffer.append(createTripleLocation(eventURI, locationID))
        f.writelines(filebuffer)
    f.close()
    print("Program Finished!")
    time_elapsed = datetime.now() - start_time 
    print('Time elapsed (hh:mm:ss) {}'.format(time_elapsed))

In [7]:
registrations_csv_path = "../../marriages/Registrations/registrations-all.csv"
output_file_registrations = "registrations.nq"
convertRegistrationsToRDF(registrations_csv_path, output_file_registrations)
persons_csv_path = "../../marriages/Persons/persons-all.csv"
output_file_persons = "persons.nq"
convertPersonsToRDF(persons_csv_path, output_file_persons)

Converting Registrations CSV file to RDF...
# 0 rows
# 10000 rows
# 20000 rows
# 30000 rows
# 40000 rows
# 50000 rows
# 60000 rows
# 70000 rows
# 80000 rows
# 90000 rows
# 100000 rows
# 110000 rows
# 120000 rows
# 130000 rows
# 140000 rows
# 150000 rows
# 160000 rows
# 170000 rows
# 180000 rows
# 190000 rows
# 200000 rows
# 210000 rows
# 220000 rows
# 230000 rows
# 240000 rows
# 250000 rows
# 260000 rows
# 270000 rows
# 280000 rows
# 290000 rows
# 300000 rows
# 310000 rows
# 320000 rows
# 330000 rows
# 340000 rows
# 350000 rows
# 360000 rows
# 370000 rows
# 380000 rows
# 390000 rows
# 400000 rows
# 410000 rows
# 420000 rows
# 430000 rows
# 440000 rows
# 450000 rows
# 460000 rows
# 470000 rows
# 480000 rows
# 490000 rows
# 500000 rows
# 510000 rows
# 520000 rows
# 530000 rows
# 540000 rows
# 550000 rows
# 560000 rows
# 570000 rows
# 580000 rows
# 590000 rows
# 600000 rows
# 610000 rows
# 620000 rows
# 630000 rows
# 640000 rows
# 650000 rows
# 660000 rows
# 670000 rows
# 680000 rows
# 69

# 1270000 rows
# 1280000 rows
# 1290000 rows
# 1300000 rows
# 1310000 rows
# 1320000 rows
# 1330000 rows
# 1340000 rows
# 1350000 rows
# 1360000 rows
# 1370000 rows
# 1380000 rows
# 1390000 rows
# 1400000 rows
# 1410000 rows
# 1420000 rows
# 1430000 rows
# 1440000 rows
# 1450000 rows
# 1460000 rows
# 1470000 rows
# 1480000 rows
# 1490000 rows
# 1500000 rows
Role number: 3 - Certificate ID: 36872325
# 1510000 rows
# 1520000 rows
Role number: 11 - Certificate ID: 36876297
# 1530000 rows
# 1540000 rows
# 1550000 rows
# 1560000 rows
# 1570000 rows
# 1580000 rows
# 1590000 rows
# 1600000 rows
Role number: 3 - Certificate ID: 36889877
Role number: 3 - Certificate ID: 36889877
Role number: 2 - Certificate ID: 36889877
Role number: 2 - Certificate ID: 36889877
Role number: 99 - Certificate ID: 36889877
Role number: 99 - Certificate ID: 36889877
# 1610000 rows
# 1620000 rows
# 1630000 rows
# 1640000 rows
# 1650000 rows
# 1660000 rows
# 1670000 rows
# 1680000 rows
# 1690000 rows
# 1700000 rows
#

# 4250000 rows
Role number: 3 - Certificate ID: 37631931
Role number: 2 - Certificate ID: 37631931
Role number: 3 - Certificate ID: 37632516
Role number: 2 - Certificate ID: 37632516
# 4260000 rows
Role number: 3 - Certificate ID: 37634626
Role number: 2 - Certificate ID: 37634626
# 4270000 rows
Role number: 3 - Certificate ID: 37636195
Role number: 2 - Certificate ID: 37636195
# 4280000 rows
# 4290000 rows
Role number: 3 - Certificate ID: 37639092
Role number: 2 - Certificate ID: 37639092
# 4300000 rows
# 4310000 rows
# 4320000 rows
Role number: 3 - Certificate ID: 37643438
Role number: 2 - Certificate ID: 37643438
# 4330000 rows
# 4340000 rows
# 4350000 rows
# 4360000 rows
Role number: 3 - Certificate ID: 37651035
Role number: 2 - Certificate ID: 37651035
# 4370000 rows
# 4380000 rows
# 4390000 rows
# 4400000 rows
# 4410000 rows
Role number: 3 - Certificate ID: 37660003
Role number: 2 - Certificate ID: 37660003
# 4420000 rows


b'Skipping line 4420378: expected 132 fields, saw 133\nSkipping line 4420381: expected 132 fields, saw 133\n'


Role number: 3 - Certificate ID: 37660315
Role number: 2 - Certificate ID: 37660315
# 4430000 rows
Role number: 3 - Certificate ID: 37662006
Role number: 2 - Certificate ID: 37662006
Role number: 3 - Certificate ID: 37663313
Role number: 2 - Certificate ID: 37663313
# 4440000 rows
Role number: 99 - Certificate ID: 37664728
# 4450000 rows
# 4460000 rows
# 4470000 rows
Role number: 3 - Certificate ID: 37669837
Role number: 2 - Certificate ID: 37669837
# 4480000 rows
# 4490000 rows
Role number: 3 - Certificate ID: 37672206
Role number: 2 - Certificate ID: 37672206
# 4500000 rows
# 4510000 rows
# 4520000 rows
Role number: 3 - Certificate ID: 37677097
Role number: 2 - Certificate ID: 37677097
# 4530000 rows
Role number: 3 - Certificate ID: 37679505
Role number: 2 - Certificate ID: 37679505
# 4540000 rows
# 4550000 rows
# 4560000 rows
# 4570000 rows
# 4580000 rows
# 4590000 rows
# 4600000 rows
# 4610000 rows
# 4620000 rows
# 4630000 rows
# 4640000 rows
# 4650000 rows
# 4660000 rows
# 4670000

...

Role number: 12 - Certificate ID: 66335185
Role number: 12 - Certificate ID: 66335194
Role number: 12 - Certificate ID: 66335208
Role number: 12 - Certificate ID: 66335265
Role number: 12 - Certificate ID: 66335265
Role number: 12 - Certificate ID: 66335273
Role number: 12 - Certificate ID: 66335273
Role number: 12 - Certificate ID: 66335276
Role number: 12 - Certificate ID: 66335276
Role number: 12 - Certificate ID: 66335296
Role number: 12 - Certificate ID: 66335332
Role number: 12 - Certificate ID: 66335375
Role number: 12 - Certificate ID: 66335418
Role number: 12 - Certificate ID: 66335422
Role number: 12 - Certificate ID: 66335440
Role number: 12 - Certificate ID: 66335506
Role number: 12 - Certificate ID: 66335536
Role number: 12 - Certificate ID: 66335538
Role number: 12 - Certificate ID: 66335540
Role number: 12 - Certificate ID: 66335548
Role number: 12 - Certificate ID: 66335571
Role number: 12 - Certificate ID: 66335605
Role number: 12 - Certificate ID: 66335605
Role number

Role number: 12 - Certificate ID: 66338773
Role number: 12 - Certificate ID: 66338773
Role number: 12 - Certificate ID: 66338784
Role number: 12 - Certificate ID: 66338810
Role number: 12 - Certificate ID: 66338820
Role number: 12 - Certificate ID: 66338851
Role number: 12 - Certificate ID: 66338851
Role number: 12 - Certificate ID: 66338882
Role number: 12 - Certificate ID: 66338945
Role number: 12 - Certificate ID: 66338979
Role number: 12 - Certificate ID: 66338979
Role number: 12 - Certificate ID: 66338979
Role number: 12 - Certificate ID: 66339002
Role number: 12 - Certificate ID: 66339022
Role number: 12 - Certificate ID: 66339038
Role number: 12 - Certificate ID: 66339040
Role number: 12 - Certificate ID: 66339059
Role number: 12 - Certificate ID: 66339082
Role number: 12 - Certificate ID: 66339099
Role number: 12 - Certificate ID: 66339099
Role number: 12 - Certificate ID: 66339123
Role number: 12 - Certificate ID: 66339150
Role number: 12 - Certificate ID: 66339161
Role number

Role number: 12 - Certificate ID: 66342148
Role number: 12 - Certificate ID: 66342148
Role number: 12 - Certificate ID: 66342198
Role number: 12 - Certificate ID: 66342223
Role number: 12 - Certificate ID: 66342259
Role number: 12 - Certificate ID: 66342268
Role number: 12 - Certificate ID: 66342318
Role number: 12 - Certificate ID: 66342323
Role number: 12 - Certificate ID: 66342333
Role number: 12 - Certificate ID: 66342342
Role number: 12 - Certificate ID: 66342346
Role number: 12 - Certificate ID: 66342358
Role number: 12 - Certificate ID: 66342362
Role number: 12 - Certificate ID: 66342366
Role number: 12 - Certificate ID: 66342370
Role number: 12 - Certificate ID: 66342374
Role number: 12 - Certificate ID: 66342380
Role number: 12 - Certificate ID: 66342380
Role number: 12 - Certificate ID: 66342396
Role number: 12 - Certificate ID: 66342398
Role number: 12 - Certificate ID: 66342414
Role number: 12 - Certificate ID: 66342416
Role number: 12 - Certificate ID: 66342419
Role number

Role number: 12 - Certificate ID: 66345239
Role number: 12 - Certificate ID: 66345276
Role number: 12 - Certificate ID: 66345314
Role number: 12 - Certificate ID: 66345329
Role number: 12 - Certificate ID: 66345422
Role number: 12 - Certificate ID: 66345506
Role number: 12 - Certificate ID: 66345506
Role number: 12 - Certificate ID: 66345508
Role number: 12 - Certificate ID: 66345542
Role number: 12 - Certificate ID: 66345542
Role number: 12 - Certificate ID: 66345544
Role number: 12 - Certificate ID: 66345544
Role number: 12 - Certificate ID: 66345545
Role number: 12 - Certificate ID: 66345545
Role number: 12 - Certificate ID: 66345556
Role number: 12 - Certificate ID: 66345583
Role number: 12 - Certificate ID: 66345587
Role number: 12 - Certificate ID: 66345599
Role number: 12 - Certificate ID: 66345600
Role number: 12 - Certificate ID: 66345600
Role number: 12 - Certificate ID: 66345601
Role number: 12 - Certificate ID: 66345609
Role number: 12 - Certificate ID: 66345614
Role number

Role number: 12 - Certificate ID: 66348564
Role number: 12 - Certificate ID: 66348637
Role number: 12 - Certificate ID: 66348637
Role number: 12 - Certificate ID: 66348651
Role number: 12 - Certificate ID: 66348680
Role number: 12 - Certificate ID: 66348686
Role number: 12 - Certificate ID: 66348687
Role number: 12 - Certificate ID: 66348744
Role number: 12 - Certificate ID: 66348752
Role number: 12 - Certificate ID: 66348769
Role number: 12 - Certificate ID: 66348770
Role number: 12 - Certificate ID: 66348876
Role number: 12 - Certificate ID: 66348876
Role number: 12 - Certificate ID: 66348903
Role number: 12 - Certificate ID: 66348903
Role number: 12 - Certificate ID: 66348907
Role number: 12 - Certificate ID: 66348969
Role number: 12 - Certificate ID: 66348978
Role number: 12 - Certificate ID: 66348981
Role number: 12 - Certificate ID: 66349039
Role number: 12 - Certificate ID: 66349057
Role number: 12 - Certificate ID: 66349069
Role number: 12 - Certificate ID: 66349069
Role number